In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [3]:
# already sorted by session, timestamp
df = pd.read_parquet("input/transactions.parquet")
dtypes = dict(df.dtypes)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216716096 entries, 0 to 216716095
Data columns (total 4 columns):
 #   Column   Dtype
---  ------   -----
 0   session  int32
 1   aid      int32
 2   ts       int64
 3   type     int8 
dtypes: int32(2), int64(1), int8(1)
memory usage: 3.4 GB


In [4]:
vocab: Dict[str, int] = {
    "<s>": 0,
    "<pad>": 1,
    "</s>": 2,
    "<unk>": 3,
    "<mask>": 4,
    "<click>": 5,
    "<cart>": 6,
    "<order>": 7,
}
_next = 8
prev = None
input_ids = []
rows = []
for t in tqdm(df.itertuples()):
    curr = int(getattr(t, "session"))
    if prev is not None and prev!=curr and len(input_ids)!=0:
        rows.append({
            "session": prev,
            "seq": " ".join(str(x) for x in input_ids),
            "length": len(input_ids),
        })
        input_ids = []
    prev = curr
    _type = int(getattr(t, "type"))
    first = vocab["<click>"]
    if _type==1:
        first = vocab["<cart>"]
    elif _type==2:
        first = vocab["<order>"]
    aid = str(getattr(t, "aid"))
    if aid not in vocab:
        vocab[aid] = _next
        _next += 1
    second = vocab[aid]
    input_ids += [first, second]
if len(input_ids)!=0:
    rows.append({
        "session": prev,
        "seq": " ".join(str(x) for x in input_ids),
        "length": len(input_ids),
    })

74950114it [03:35, 348013.90it/s]


KeyboardInterrupt: 

In [ ]:
with open("output/vocab.json", "w") as f:
    json.dump(vocab, f)

In [ ]:
df = pd.DataFrame.from_records(rows)
df = df.astype(dtypes)
cols = ["length"]
df[cols] = df[cols].astype(np.int16)
df.info()

In [ ]:
df.describe(percentiles=percentiles)

In [ ]:
df.head()

In [ ]:
%%time
assert df.notna().all(axis=None)
df.to_parquet("output/sequences.parquet", index=False)